# Investigating the co-evolution of cooperation and intelligence using Neuroevolution of Augmenting Topologies (Removed Fitness Penalty for Intelligence)

### Setup

Importing all the relevant libraries for the simulations and analyses:
 - `dill` - is used to  save session data due to long run times.
 - `random` - used to generate random numbers.
 - `neat` - the Neat-Python library used to generate the population of neural networks and evolve them using the NEAT aglorithm alongside a fitness function.
 - `numpy` - one of the most popular numerical computing tools for Python.
 - `pandas` - used to construct and edit dataframes for the collected data.
 - `statistics` - the `covariance` function is used from the basic statistics library in Python to help calculate the selection for intelligence.
 - `scipy.stats` - the `spearmanr` function is used here to carry out Spearman's rank corrleation tests on the collected data.
 - `matplotlib` and `seaborn` - both of these libraries are used for all figure construction/data visualisaiton. 
 - `statmodels` - used to calculate lowess lines for correlation plots. 

In [ ]:
# Import relevant libraries
import dill
import random
import neat
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from scipy.stats import rankdata
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [ ]:
# Define all main settings for the model outside the config file
# Payoffs - R = Reward, P = Punishment, T = Temptation, S = Sucker
# IPD
IPD_R = 6
IPD_P = 2
IPD_T = 7
IPD_S = 1

# ISD
ISD_R = 5
ISD_P = 1
ISD_T = 8
ISD_S = 2

# Number of rounds per interaction (number of rounds for each iterated game)
num_rounds = 50

# Number of rounds per test
test_num_rounds = 20

# Number of generations to simulate
num_generations = 10000

# Cooperation threshold - above what output should cooperation occur - here if above 0.5, assume 1 (cooperate); if bellow 0.5, assume 0 (defect)
cooperation_threshold = 0.5

# Cost of intelligence - the value to multiply the intelligence metric by to find the fitness penalty of intelligence
intelligence_cost = 0

# Create lists to store the results for each generation of the model
# IPD
IPD_network_fitness_values = []
IPD_mean_fitness = []
IPD_network_intelligence_values = []
IPD_mean_intelligence = []
IPD_cooperation_frequency = []
IPD_strategies_each_generation = []

# ISD
ISD_network_fitness_values = []
ISD_mean_fitness = []
ISD_network_intelligence_values = []
ISD_mean_intelligence = []
ISD_cooperation_frequency = []
ISD_strategies_each_generation = []

### Test-set Generation

In [ ]:
def generate_test_set(test_num_rounds):
    # Opponents' cooperation probabilities - ranging from 0 to 1 in increments of 0.25
    cooperation_probs = [0, 0.25, 0.5, 0.75, 1]

    # List to store each list of moves for each opponent
    test_set = []

    # Generate moves for each opponent
    for prob in cooperation_probs:
        opponent_moves = []
        for round in range(test_num_rounds):
            move = 1 if random.random() < prob else 0  # Cooperate (1) with probability prob, otherwise defect (0)
            opponent_moves.append(move)
        test_set.append(opponent_moves)

    return test_set

test_set_moves = generate_test_set(test_num_rounds)
print(test_set_moves)


### Pure Strategy Generation

Functions which take in the test-set of moves, and generate the moves each pure strategy would make against the test-set moves:

*Always Cooperate*

In [ ]:
def always_cooperate_strategy(test_set):
    # List to store always cooperate moves
    all_always_cooperate_moves = []
    
    # Generate a set of moves the same size of each test-set which are always 1
    for opponent_moves in test_set:
        always_cooperate_moves = []
        for move in opponent_moves:
            always_cooperate_moves.append(1)
        all_always_cooperate_moves.append(always_cooperate_moves)

    return all_always_cooperate_moves

# Generate the always cooperate moves (always 1)
always_cooperate_moves = always_cooperate_strategy(test_set_moves)
print(always_cooperate_moves)

*Always Defect*

In [ ]:
def always_defect_strategy(test_set):
    # List to store always defect moves
    all_always_defect_moves = []
    
    # Generate a set of moves the same size of each test-set which are always 0
    for opponent_moves in test_set:
        always_defect_moves = []
        for move in opponent_moves:
            always_defect_moves.append(0)
        all_always_defect_moves.append(always_defect_moves)

    return all_always_defect_moves

# Generate the always defect moves (always 0)
always_defect_moves = always_defect_strategy(test_set_moves)
print(always_defect_moves)

*Tit-for-tat*

In [ ]:
def tit_for_tat_strategy(test_set):
    # List to store the moves played by the tit-for-tat strategy against all test set moves
    all_tit_for_tat_moves = []

    # Play against each test-set opponents moves
    for opponent_moves in test_set:
        tit_for_tat_moves = []
        last_move = 1  # Start by cooperating in the first round
        for move in opponent_moves:
            tit_for_tat_moves.append(last_move)
            last_move = move  # Mirrors the opponent's last move in the next round
        all_tit_for_tat_moves.append(tit_for_tat_moves)

    return all_tit_for_tat_moves

# Generate the tit-for-tat moves
tit_for_tat_moves = tit_for_tat_strategy(test_set_moves)
print(tit_for_tat_moves)

*Tit-for-two-tats*

In [ ]:
def tit_for_two_tats_strategy(test_set):
    # List to store the moves played by the tit-for-two-tats strategy against all test set moves
    all_tit_for_two_tats_moves = []

    # Play against each test-set opponents moves
    for opponent_moves in test_set:
        tit_for_two_tats_moves = []
        consecutive_defects = 0
        for move in opponent_moves:
            if consecutive_defects < 2:
                # Cooperate if the opponent has defected fewer than 2 consecutive times
                tit_for_two_tats_moves.append(1)
                if move == 0:
                    consecutive_defects += 1
                else:
                    consecutive_defects = 0
            else:
                # Defect if the opponent has defected two or more consecutive times
                tit_for_two_tats_moves.append(0)
        all_tit_for_two_tats_moves.append(tit_for_two_tats_moves)

    return all_tit_for_two_tats_moves

# Generate tit-for-two-tats moves
tit_for_two_tats_moves = tit_for_two_tats_strategy(test_set_moves)
print(tit_for_two_tats_moves)

*Pavlov*

In [ ]:
def pavlov_strategy(test_set):
    # List to store the moves played by the pavlov strategy against all test set moves
    all_pavlov_moves =[]

    # Play against each test-set opponents moves
    for opponents_moves in test_set:
        pavlov_moves = []
        
        # Initialise previous move variables
        pavlov_last_move = 1
        opponent_last_move = 1

        for move in opponents_moves:
            # Win Stay - mutual cooperation or pavlov exploits
            if (pavlov_last_move == 1 and opponent_last_move == 1) or ((pavlov_last_move == 0 and opponent_last_move == 1)):
                pavlov_moves.append(pavlov_last_move)
            # Lose Shift - mutual defection, or pavlov cooperates, test exploits
            else:
                pavlov_moves.append(1 - pavlov_last_move) # Switch moves (1-0 = 1, 1-1 = 0)
                pavlov_last_move = 1 - pavlov_last_move

            # Update last moves for the test-set
            opponent_last_move = move
      
        all_pavlov_moves.append(pavlov_moves)

    return all_pavlov_moves

# Generate pavlov moves    
pavlov_moves = pavlov_strategy(test_set_moves)
print(pavlov_moves)    


### Sum of Squares

Below is a function which calculates the mean moves of each pure strategy across each test set sequence.

In [ ]:
def mean_moves(all_moves):
    num_lists = len(all_moves)
    num_elements = len(all_moves[0])

    mean_moves = []
    for i in range(num_elements):
        total= 0
        for j in range(num_lists):
            total += all_moves[j][i]
        mean = total/num_lists
        mean_moves.append(mean)
    
    return mean_moves

# Generate the mean move lists for each strategy
mean_always_cooperate_moves = mean_moves(always_cooperate_moves)
mean_always_defect_moves = mean_moves(always_defect_moves)
mean_tit_for_tat_moves = mean_moves(tit_for_tat_moves)
mean_tit_for_two_tats_moves = mean_moves(tit_for_two_tats_moves)
mean_pavlov_moves = mean_moves(pavlov_moves)

# Dictionary to store strategy move sequences
mean_strategy_moves = {
    'always_cooperate': mean_always_cooperate_moves,
    'always_defect': mean_always_defect_moves,
    'tit_for_tat': mean_tit_for_tat_moves,
    'tit_for_two_tats': mean_tit_for_two_tats_moves,
    'pavlov': mean_pavlov_moves
    }

print(mean_strategy_moves)

These mean moves can then be used to compare the similarity between strategies using the *between group sum of squares*. This will be used later to classify the emergent network's strategies to their closest pure strategy.

In [ ]:
def between_group_sum_of_squares(list1, list2):
    grand_mean = np.mean(list1 + list2)
    group_mean1 = np.mean(list1)
    group_mean2 = np.mean(list2)
    between_group_ss = len(list1) * (group_mean1 - grand_mean) ** 2 + len(list2) * (group_mean2 - grand_mean) ** 2
    return between_group_ss

print(between_group_sum_of_squares(always_cooperate_moves, always_defect_moves))
print(between_group_sum_of_squares(pavlov_moves, tit_for_tat_moves))
print(between_group_sum_of_squares(tit_for_tat_moves, tit_for_tat_moves))

In [ ]:
# Function to cluster a set of moves to the cloesest pure strategy
def cluster_strategy(mean_network_movelist):

    closest_strategy = ""
    smallest_sum_of_squares = float('inf')  # Initialize with a large value

    for strategy, mean_strategy_movelist in mean_strategy_moves.items():
        sum_of_squares = between_group_sum_of_squares(mean_network_movelist, mean_strategy_movelist)
        if smallest_sum_of_squares > sum_of_squares:
                smallest_sum_of_squares = sum_of_squares
                closest_strategy = strategy

    return closest_strategy

# Test to show it groups each strategy as as being cloest to itself - output in order should be: always_cooperate, always_defect, tit_for_tat, tit_for_two_tats and pavlov
# As per the mean_strategy_moves dictionary
for strategy, mean_strategy_movelist in mean_strategy_moves.items():
     print(cluster_strategy(mean_strategy_movelist))

### NEAT Simulation

In [ ]:
# Step 1: Set up the NEAT configuration
# Define the configuration file path and load it
config_path = 'config_relaxed.txt'
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)

 `nodes` in the genome module refers to a dictionary containing the information concerning nodes for that particular genome. The first item in the dictionary ALWAYS corresponds to the output node and the input nodes are NOT included in this dictionary (see source code for genome). Therefore to obtain the number of hidden nodes (evolved nodes) in the network we return the length of the dictionary - 1. If we wanted to summarise intelligence as the TOTAL number of nodes, we would simply change -1 to +3.

In [ ]:
# Step 2: Define a function to calculate the intelligence metric
# Here it is just the number of hidden nodes present in the neural network 
def calculate_intelligence(genome):
    return len(genome.nodes)-1

In [ ]:
# Step 3: Function to calculate payoffs based on player's and opponent's moves - used to simplfy later code
def calculate_payoffs(player_move, opponent_move, cooperation_threshold, P, T, S, R):

    # Initilaise the cooperative act variable (returns as 0 if no cooperation, 1 if there has been a cooperative act from the PLAYER)
    cooperative_act = 0

    if player_move < cooperation_threshold:
        if opponent_move < cooperation_threshold:  # Both Defect
            player_score = P
            opponent_score = P
        else:  # Player Defects, Opponent Cooperates
            player_score = T
            opponent_score = S
            
    else:
        if opponent_move < cooperation_threshold:  # Player Cooperates, Opponent Defects
            player_score = S
            opponent_score = T

            cooperative_act = 1
        
        else:  # Both Cooperate
            player_score = R
            opponent_score = R

            cooperative_act = 1

    return player_score, opponent_score, cooperative_act

In [ ]:
# Step 4: Function to play rounds of the game in the fitness calculating stage
def play_rounds(network, opponent_network, num_rounds, cooperation_threshold, P, T, S, R):
    # Initialise score and cooperative act count variables
    player_score, opponent_score = 0, 0
    total_player_score = 0
    cooperative_acts = 0
    round_number = 0

    # Play game for the number of rounds defined
    for _ in range(num_rounds):
        # If it is the first round, randomly determine first moves
        if round_number == 0:
            player_move = random.randint(0, 1)
            opponent_move = random.randint(0, 1)
        else:
            # Activate the player and opponent networks to calculate the moves
            player_move = network.activate([player_score, opponent_score])[0]
            opponent_move = opponent_network.activate([opponent_score, player_score])[0]
        
        # Use the calculate_payoff function to return the scores for this round and if there was a cooperative act or not (0 or 1)
        player_score, opponent_score, cooperative_act = calculate_payoffs(player_move, opponent_move, cooperation_threshold, P, T, S, R)
        
        # Add the score and acts for this round to the totals for the interaction
        total_player_score += player_score
        cooperative_acts += cooperative_act
        
        # Increment round number
        round_number += 1

    return total_player_score, cooperative_acts

In [ ]:
# Step 5: Define the fitness function - most important function here, represents one generation of interactions
# IPD fitness function
def eval_genomes_IPD(genomes, config):

    # EVOLVING / FITNESS CALCULATION
    # Initialise result variables for each generation
    generation_total_cooperative_acts = 0
    generation_total_fitness = 0
    generation_total_intelligence = 0
    generation_total_interactions = 0
    generation_network_fitness_values = []
    generation_network_intelligence_values = []
    
    # For each genome in the population
    for genome_id, genome in genomes:
        # Retrieve the network from the genome
        player_network = neat.nn.RecurrentNetwork.create(genome, config)

        # Initialise fitness of the genome
        genome_total_fitness = 0.0

        # Intiialise cooperative acts for this genome
        cooperative_acts = 0

        # Retrieve the intelligence metric for the current genome and increase the total intelligence for the generation
        intelligence_metric = calculate_intelligence(genome)
        generation_network_intelligence_values.append(intelligence_metric)
        generation_total_intelligence += intelligence_metric

        # Calculate the intelligence penalty for this genome/network
        intelligence_penalty = intelligence_metric * intelligence_cost

        # Play every other genome in the population
        for opponent_genome_id, opponent_genome in genomes:
            # Skip playing against itself
            if genome_id == opponent_genome.key:
                continue

            # Retrieve the opponent network
            opponent_network = neat.nn.RecurrentNetwork.create(opponent_genome, config)

            # Initialise variables
            total_player_score = 0

            # Play multiple rounds of the game (defined by num_rounds) - IPD payoffs
            total_player_score, cooperative_acts = play_rounds(player_network, opponent_network, num_rounds, cooperation_threshold, IPD_P, IPD_T, IPD_S, IPD_R)

            # Increase total interactions dependant on the number of rounds being played
            generation_total_interactions += num_rounds

            # Calculate fitness based on player's accumulated score
            genome_total_fitness += total_player_score / num_rounds

            # Add the number of cooperative acts this genome to the total count
            generation_total_cooperative_acts += cooperative_acts

        # Calculate mean fitness over all opponents
        genome_mean_fitness_per_round = genome_total_fitness / (len(genomes) - 1)

        # Apply the intelligence penalty
        genome_mean_fitness_per_round -= intelligence_penalty

        # Set the genome's fitness for neat-python
        genome.fitness = genome_mean_fitness_per_round
        
        # Add the network's fitness value to the list for this generation
        generation_network_fitness_values.append(genome_mean_fitness_per_round)

        # Add the network's fitness value to the total for this generation
        generation_total_fitness += genome_mean_fitness_per_round
    
    # Add this generation's networks intelligence and fitness values to their respective lists
    IPD_network_fitness_values.append(generation_network_fitness_values)
    IPD_network_intelligence_values.append(generation_network_intelligence_values) 
    
    # Calculate the proportion of cooperative acts by the player (frequency of cooperation) for this generation and add it to the results
    IPD_cooperation_frequency.append(generation_total_cooperative_acts / generation_total_interactions)

    # Calculate the mean fitness and intelligence across this generation
    IPD_mean_fitness.append(generation_total_fitness/len(genomes))
    IPD_mean_intelligence.append(generation_total_intelligence/len(genomes))

    #TESTING / PHENOTYPING
    # Intialise result variables for phenotyping
    # List to store all the closest pure strategies this generation
    strategies_this_generation = []

    for genome_id in genomes:
        # Retrieve the network from the genome
        network = neat.nn.RecurrentNetwork.create(genome, config)

        all_player_moves = []

        for moves in test_set_moves:
            
            # Initialise variables
            player_score = 0
            opponent_score = 0
            
            # Stores the networks move sequence for this test set sequence
            player_moves = []

            round_number = 0
            
            for move in moves:
                if round_number == 0:
                    player_move = random.randint(0, 1)
                else:
                    # Network takes its own and opponent's previous scores
                    player_move = network.activate([player_score, opponent_score])[0]
                
                opponent_move = move

                # Determine payoffs based on player's and opponent's moves
                if player_move < cooperation_threshold:
                    
                    # Set this rounds player sequence to defect
                    player_moves.append(0)

                    if opponent_move < cooperation_threshold:  # Both Defect
                        player_score = IPD_P
                        opponent_score = IPD_P
                    else:  # Player Defects, Opponent Cooperates
                        player_score = IPD_T
                        opponent_score = IPD_S
                else:
                    # Set this rounds player sequence to cooperate
                    player_moves.append(1)

                    if opponent_move < cooperation_threshold:  # Player Cooperates, Opponent Defects
                        player_score = IPD_S
                        opponent_score = IPD_T
                    else:  # Both Cooperate
                        player_score = IPD_R
                        opponent_score = IPD_R
                
                round_number += 1

            all_player_moves.append(player_moves)

        # Identify strategy
        mean_player_moves = mean_moves(all_player_moves)
        strategies_this_generation.append(cluster_strategy(mean_player_moves))

    # Add the strategies observed this generation to the list for each generation
    IPD_strategies_each_generation.append(strategies_this_generation)


In [ ]:
# Step 6: Define the fitness function for ISD - same as above however applies the ISD payoffs instead
def eval_genomes_ISD(genomes, config):

    # EVOLVING / FITNESS CALCULATION
    # Initialise result variables for each generation
    generation_total_cooperative_acts = 0
    generation_total_fitness = 0
    generation_total_intelligence = 0
    generation_total_interactions = 0
    generation_network_fitness_values = []
    generation_network_intelligence_values = []
    
    for genome_id, genome in genomes:
        # Retrieve the network from the genome
        player_network = neat.nn.RecurrentNetwork.create(genome, config)

        # Initialise fitness of the genome
        genome_total_fitness = 0.0

        # Intiialise cooperative acts for this genome
        cooperative_acts = 0

        # Retrieve the intelligence metric for the current genome and increase the total intelligence for the generation
        intelligence_metric = calculate_intelligence(genome)
        generation_network_intelligence_values.append(intelligence_metric)
        generation_total_intelligence += intelligence_metric

        # Calculate the intelligence penalty for this genome/network
        intelligence_penalty = intelligence_metric * intelligence_cost
        
        for opponent_genome_id, opponent_genome in genomes:
            # Skip playing against itself
            if genome_id == opponent_genome.key:
                continue

            opponent_network = neat.nn.RecurrentNetwork.create(opponent_genome, config)

            # Initialise variables
            total_player_score = 0

            # Play multiple rounds of the game (defined by num_rounds) - ISD payoffs
            total_player_score, cooperative_acts = play_rounds(player_network, opponent_network, num_rounds, cooperation_threshold, ISD_P, ISD_T, ISD_S, ISD_R)

            # Increase total interactions dependant on the number of rounds being played
            generation_total_interactions += num_rounds

            # Calculate fitness based on player's accumulated score - mean payoff per round
            genome_total_fitness += total_player_score / num_rounds

            # Add the number of cooperative acts this genome to the total count
            generation_total_cooperative_acts += cooperative_acts

        # Calculate mean fitness over all opponents
        genome_mean_fitness_per_round = genome_total_fitness / (len(genomes) - 1)

        # Apply the intelligence penalty
        genome_mean_fitness_per_round -= intelligence_penalty

        # Set the genome's fitness for neat-python
        genome.fitness = genome_mean_fitness_per_round
        
        # Add the network's fitness value to the list for this generation
        generation_network_fitness_values.append(genome_mean_fitness_per_round)

        # Add the network's fitness value to the total for this generation
        generation_total_fitness += genome_mean_fitness_per_round
    
    # Add this generation's networks intelligence and fitness values to their respective lists
    ISD_network_fitness_values.append(generation_network_fitness_values)
    ISD_network_intelligence_values.append(generation_network_intelligence_values) 

    # Calculate the proportion of cooperative acts by the player (frequency of cooperation) for this generation and add it to the results
    ISD_cooperation_frequency.append(generation_total_cooperative_acts / generation_total_interactions)

    # Calculate the mean fitness and intelligence across this generation
    ISD_mean_fitness.append(generation_total_fitness/len(genomes))
    ISD_mean_intelligence.append(generation_total_intelligence/len(genomes))

    #TESTING / PHENOTYPING
    # Intialise result variables for phenotyping
    # List to store all the closest pure strategies this generation
    strategies_this_generation = []

    for genome_id in genomes:
        # Retrieve the network from the genome
        network = neat.nn.RecurrentNetwork.create(genome, config)

        all_player_moves = []

        for moves in test_set_moves:
            
            # Initialise variables
            player_score = 0
            opponent_score = 0
            # Stores the networks move sequence for this test set sequence
            player_moves = []

            # Counts the round number
            round_number = 0
            
            for move in moves:
                if round_number == 0:
                    player_move = random.randint(0, 1)
                else:
                    player_move = network.activate([player_score, opponent_score])[0]
                
                opponent_move = move

                # Determine payoffs based on player's and opponent's moves
                if player_move < cooperation_threshold:
                    
                    # Set this rounds player sequence to defect
                    player_moves.append(0)

                    if opponent_move < cooperation_threshold:  # Both Defect
                        player_score = ISD_P
                        opponent_score = ISD_P
                    else:  # Player Defects, Opponent Cooperates
                        player_score = ISD_T
                        opponent_score = ISD_S
                else:
                    # Set this rounds player sequence to cooperate
                    player_moves.append(1)

                    if opponent_move < cooperation_threshold:  # Player Cooperates, Opponent Defects
                        player_score = ISD_S
                        opponent_score = ISD_T
                    else:  # Both Cooperate
                        player_score = ISD_R
                        opponent_score = ISD_R
                
                round_number += 1
                
            all_player_moves.append(player_moves)

        # Identify strategy
        mean_player_moves = mean_moves(all_player_moves)
        strategies_this_generation.append(cluster_strategy(mean_player_moves))

    # Add the strategies observed this generation to the list for each generation
    ISD_strategies_each_generation.append(strategies_this_generation)

In [ ]:
# Step 7: Create the NEAT population
def create_population(config):
    population = neat.Population(config)
    return population

# Step 8: Function to run the simulation and report the progress
def run_neat_simulation(game, config):
    population = create_population(config)
    population.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    population.add_reporter(stats)

    # Run the model passing the fitness function defined and the number of generations to run for
    if game == "IPD":
        population.run(eval_genomes_IPD, num_generations)
    if game == "ISD":
        population.run(eval_genomes_ISD, num_generations)

In [ ]:
# Step 9: Run the NEAT simulation for the IPD
run_neat_simulation("IPD", config)

# Step 10: Run again for the ISD
run_neat_simulation("ISD", config)

# Save the session incase of crash
dill.dump_session('session_save_relaxed.db')

### IPD Analysis

*Load the session if required*

In [ ]:
dill.load_session('session_save_relaxed.db')

*Create a Pandas Data Frame to Store all Results*

In [ ]:
# Create a data library
IPD_data = {
    "IPD_network_fitness_values" : IPD_network_fitness_values,
    "IPD_mean_fitness" : IPD_mean_fitness,
    "IPD_network_intelligence_values" : IPD_network_intelligence_values,
    "IPD_mean_intelligence" : IPD_mean_intelligence,
    "IPD_cooperation_frequency" : IPD_cooperation_frequency,
    "IPD_strategies_each_generation" : IPD_strategies_each_generation,
}

# Create a pandas dataframe from the data library
IPD_df = pd.DataFrame(IPD_data)

# Make the index the generation number
IPD_df.index.name = 'Generation'

# Make the index (generation) start at 1 instead of 0
IPD_df.index = IPD_df.index + 1 

IPD_df.head()

*IPD Change in Intelligence and Cooperation Frequency Plots*

In [ ]:
# Setting the seaborn theme
sns.set_theme()
# Plotting the change in mean intelligence over generations
plt.figure(figsize=(10, 5))
sns.lineplot(data = IPD_df, x= 'Generation', y= 'IPD_mean_intelligence')
plt.xlabel('Generations')
plt.ylabel('Mean Intelligence')
plt.title('Change in Mean Intelligence over Generations')
plt.grid(True)
plt.show()

# Plotting the change in frequency of cooperation over generations
plt.figure(figsize=(10, 5))
sns.lineplot(data = IPD_df, x= 'Generation', y= 'IPD_cooperation_frequency')
plt.xlabel('Generations')
plt.ylabel('Frequency of Cooperation')
plt.title('Change in Frequency of Cooperation over Generations')
plt.grid(True)

plt.show()

*IPD Overall Mean Intelligence and Frequency of Cooperation*

In [ ]:
# Mean intelligence
print("Overall Mean Intellgence:")
print(IPD_df['IPD_mean_intelligence'].mean())

# Mean frequency of cooperation
print("Mean Frequency of Cooperation:")
print(IPD_df['IPD_cooperation_frequency'].mean())

*IPD Frequency of Strategies across Generations*

In [ ]:
# Extract unique strategies from IPD_strategies_each_generation
unique_strategies = set(strategy for strategies in IPD_df["IPD_strategies_each_generation"] for strategy in strategies)

# Calculate strategy frequencies for each generation
for strategy in unique_strategies:
    IPD_df[strategy + "_frequency"] = IPD_df["IPD_strategies_each_generation"].apply(lambda x: x.count(strategy) / len(x))

IPD_df.head()

In [ ]:
plt.figure(figsize=(12, 8))

# Define the rolling window
rolling_window = 10

# Apply rolling mean for smoother visualization
rolling_always_cooperate = IPD_df['always_cooperate_frequency'].rolling(window=rolling_window).mean()
rolling_always_defect = IPD_df['always_defect_frequency'].rolling(window=rolling_window).mean()
rolling_tit_for_tat = IPD_df['tit_for_tat_frequency'].rolling(window=rolling_window).mean()
rolling_tit_for_two_tats = IPD_df['tit_for_two_tats_frequency'].rolling(window=rolling_window).mean()
rolling_pavlov = IPD_df['pavlov_frequency'].rolling(window=rolling_window).mean()

plt.subplot(2, 3, 1)
sns.lineplot(data=IPD_df, x='Generation', y=rolling_always_cooperate)
plt.xlabel('Generations')
plt.ylabel('Frequency')
plt.title('Always Cooperate')
plt.grid(True)
plt.ylim(0, 1)

plt.subplot(2, 3, 2)
sns.lineplot(data=IPD_df, x='Generation', y=rolling_always_defect, color='#dd8452')
plt.xlabel('Generations')
plt.ylabel('Frequency')
plt.title('Always Defect')
plt.grid(True)
plt.ylim(0, 1)

plt.subplot(2, 3, 3)
sns.lineplot(data=IPD_df, x='Generation', y=rolling_tit_for_tat, color='#58ac6c')
plt.xlabel('Generations')
plt.ylabel('Frequency')
plt.title('Tit-for-Tat')
plt.grid(True)
plt.ylim(0, 1)

plt.subplot(2, 3, 4)
sns.lineplot(data=IPD_df, x='Generation', y=rolling_tit_for_two_tats, color='#c84c54')
plt.xlabel('Generations')
plt.ylabel('Frequency')
plt.title('Tit-for-Two-Tats')
plt.grid(True)
plt.ylim(0, 1)

plt.subplot(2, 3, 5)
sns.lineplot(data=IPD_df, x='Generation', y=rolling_pavlov, color='#e08cc4')
plt.xlabel('Generations')
plt.ylabel('Frequency')
plt.title('Pavlov')
plt.grid(True)
plt.ylim(0, 1)

plt.tight_layout()  # Adjust layout for better spacing
plt.show()

*IPD Mean Strategy Frequencies*

In [ ]:
# Calculate mean strategy frequencies
IPD_mean_always_cooperate_freq = IPD_df['always_cooperate_frequency'].mean()
IPD_mean_always_defect_freq = IPD_df['always_defect_frequency'].mean()
IPD_mean_tit_for_tat_freq = IPD_df['tit_for_tat_frequency'].mean()
IPD_mean_tit_for_two_tats_freq = IPD_df['tit_for_two_tats_frequency'].mean()
IPD_mean_pavlov_freq = IPD_df['pavlov_frequency'].mean()

# Print the mean values
print("Mean always cooperate frequency:", IPD_mean_always_cooperate_freq)
print("Mean always defect frequency:", IPD_mean_always_defect_freq)
print("Mean tit for tat frequency:", IPD_mean_tit_for_tat_freq)
print("Mean tit for two tats frequency:", IPD_mean_tit_for_two_tats_freq)
print("Mean pavlov frequency:", IPD_mean_pavlov_freq)


*IPD Spearman's Rank Correlation between frequency of cooperation and mean intelligence*

In [ ]:
# Spearman's Rank Correlation for frequency of cooperative acts and mean intelligence
# Compute the correlation coefficient (rho) and p-value
IPD_intelligence_cooperation_rho, IPD_intelligence_cooperation_pvalue = spearmanr(IPD_cooperation_frequency, IPD_mean_intelligence)

print("Spearman's Rank Correlation:")
print("rho:", IPD_intelligence_cooperation_rho)
print("p-value:", IPD_intelligence_cooperation_pvalue)

In [ ]:
# Calculate the LOWESS line
lowess = sm.nonparametric.lowess(IPD_cooperation_frequency, IPD_mean_intelligence)

plt.figure(figsize=(10, 5))
sns.scatterplot(x=IPD_mean_intelligence, y=IPD_cooperation_frequency, alpha=0.5)
plt.plot(lowess[:, 0], lowess[:, 1], linewidth=2)  # Plot the LOESS line
plt.xlabel('Mean Intelligence')
plt.ylabel('Frequency of Cooperation')
plt.title('Correlation between Frequency of Cooperation and Mean Intelligence')
plt.grid(True)
plt.show()

*IPD Selection for Intelligence*

The measure for the selection for intelligence in each generation is calculated as the covariance between fitness and intelligence, divided by the mean fitness of the population in that generation.

In [ ]:
# Calculate the covariance between fitness and intelligence in each generation and store them in a list
IPD_covariance_values = []
for fitness_values, intelligence_values in zip(IPD_df['IPD_network_fitness_values'], IPD_df['IPD_network_intelligence_values']):
    cov = np.cov(fitness_values, intelligence_values)[0][1]
    IPD_covariance_values.append(cov)

# Divide by the mean fitness in the population for each generation
IPD_selection_for_intelligence = [cov / mean_fitness for cov, mean_fitness in zip(IPD_covariance_values, IPD_mean_fitness)]

# Add a column in the pandas dataframe for the selection for intelligence measure each generation.
IPD_df['IPD_selection_for_intelligence'] = IPD_selection_for_intelligence

In [ ]:
plt.figure(figsize=(10, 5))
sns.lineplot(data = IPD_df, x= 'Generation', y= 'IPD_selection_for_intelligence')
plt.xlabel('Generations')
plt.ylabel('Selection for Intelligence')
plt.title('Selection for Intelligence over Generations')
plt.grid(True)
plt.show()

In [ ]:
# Mean selection for intelligence
print(IPD_df['IPD_selection_for_intelligence'].mean())

*IPD Correlation between Selection for Intelligence and Strategy Frequencies*

In [ ]:
# Spearman's Rank Correlation for selection of intelligence and strategy frequencies
IPD_ac_selection_rho, IPD_ac_selection_pvalue = spearmanr(IPD_df['always_cooperate_frequency'], IPD_df['IPD_selection_for_intelligence'])
IPD_ad_selection_rho, IPD_ad_selection_pvalue = spearmanr(IPD_df['always_defect_frequency'], IPD_df['IPD_selection_for_intelligence'])
IPD_tft_selection_rho, IPD_tft_selection_pvalue = spearmanr(IPD_df['tit_for_tat_frequency'], IPD_df['IPD_selection_for_intelligence'])
IPD_tftt_selection_rho, IPD_tftt_selection_pvalue = spearmanr(IPD_df['tit_for_two_tats_frequency'], IPD_df['IPD_selection_for_intelligence'])
IPD_pvlv_selection_rho, IPD_pvlv_selection_pvalue = spearmanr(IPD_df['pavlov_frequency'], IPD_df['IPD_selection_for_intelligence'])

# Print results for each strategy
print("Always Cooperate:")
print("rho:", IPD_ac_selection_rho)
print("p-value:", IPD_ac_selection_pvalue)
print("")
print("Always Defect:")
print("rho:", IPD_ad_selection_rho)
print("p-value:", IPD_ad_selection_pvalue)
print("")
print("Tit for tat:")
print("rho:", IPD_tft_selection_rho)
print("p-value:", IPD_tft_selection_pvalue)
print("")
print("Tit for two tats:")
print("rho:", IPD_tftt_selection_rho)
print("p-value:", IPD_tftt_selection_pvalue)
print("")
print("Pavlov:")
print("rho:", IPD_pvlv_selection_rho)
print("p-value:", IPD_pvlv_selection_pvalue)

### ISD Analysis

*Create a Pandas Data Frame to Store the Results*

In [ ]:
# Same as for the IPD data but replacing the values with the ISD data
# Create a data library
ISD_data = {
    "ISD_network_fitness_values" : ISD_network_fitness_values,
    "ISD_mean_fitness" : ISD_mean_fitness,
    "ISD_network_intelligence_values" : ISD_network_intelligence_values,
    "ISD_mean_intelligence" : ISD_mean_intelligence,
    "ISD_cooperation_frequency" : ISD_cooperation_frequency,
    "ISD_strategies_each_generation" : ISD_strategies_each_generation,
}

# Create a pandas dataframe from the data library
ISD_df = pd.DataFrame(ISD_data)

# Make the index the generation number
ISD_df.index.name = 'Generation'

# Make the index (generation) start at 1 instead of 0
ISD_df.index = ISD_df.index + 1 

ISD_df.head()

*ISD Change in Intelligence and Cooperation Frequency Plots*

In [ ]:
# Setting the seaborn theme
sns.set_theme()
# Plotting the change in mean intelligence over generations
plt.figure(figsize=(10, 5))
sns.lineplot(data = ISD_df, x= 'Generation', y= 'ISD_mean_intelligence')
plt.xlabel('Generations')
plt.ylabel('Mean Intelligence')
plt.title('Change in Mean Intelligence over Generations')
plt.grid(True)
plt.show()

# Plotting the change in frequency of cooperation over generations
plt.figure(figsize=(10, 5))
sns.lineplot(data = ISD_df, x= 'Generation', y= 'ISD_cooperation_frequency')
plt.xlabel('Generations')
plt.ylabel('Frequency of Cooperation')
plt.title('Change in Frequency of Cooperation over Generations')
plt.grid(True)

plt.show()

*ISD Overall Mean Intelligence and Frequency of Cooperation*

In [ ]:
# Mean intelligence
print("Overall Mean Intellgence:")
print(ISD_df['ISD_mean_intelligence'].mean())

# Mean frequency of cooperation
print("Mean Frequency of Cooperation:")
print(ISD_df['ISD_cooperation_frequency'].mean())

*ISD Frequency of Strategies across Generations*

In [ ]:
# Extract unique strategies from ISD_strategies_each_generation
unique_strategies = set(strategy for strategies in ISD_df["ISD_strategies_each_generation"] for strategy in strategies)

# Calculate strategy frequencies for each generation
for strategy in unique_strategies:
    ISD_df[strategy + "_frequency"] = ISD_df["ISD_strategies_each_generation"].apply(lambda x: x.count(strategy) / len(x))

ISD_df.head()

In [ ]:
plt.figure(figsize=(12, 8))

# Define the rolling window
rolling_window = 10

# Apply rolling mean for smoother visualization
rolling_always_cooperate = ISD_df['always_cooperate_frequency'].rolling(window=rolling_window).mean()
rolling_always_defect = ISD_df['always_defect_frequency'].rolling(window=rolling_window).mean()
rolling_tit_for_tat = ISD_df['tit_for_tat_frequency'].rolling(window=rolling_window).mean()
rolling_tit_for_two_tats = ISD_df['tit_for_two_tats_frequency'].rolling(window=rolling_window).mean()
rolling_pavlov = ISD_df['pavlov_frequency'].rolling(window=rolling_window).mean()

plt.subplot(2, 3, 1)
sns.lineplot(data=ISD_df, x='Generation', y=rolling_always_cooperate)
plt.xlabel('Generations')
plt.ylabel('Frequency')
plt.title('Always Cooperate')
plt.grid(True)
plt.ylim(0, 1)

plt.subplot(2, 3, 2)
sns.lineplot(data=ISD_df, x='Generation', y=rolling_always_defect, color='#dd8452')
plt.xlabel('Generations')
plt.ylabel('Frequency')
plt.title('Always Defect')
plt.grid(True)
plt.ylim(0, 1)

plt.subplot(2, 3, 3)
sns.lineplot(data=ISD_df, x='Generation', y=rolling_tit_for_tat, color='#58ac6c')
plt.xlabel('Generations')
plt.ylabel('Frequency')
plt.title('Tit-for-Tat')
plt.grid(True)
plt.ylim(0, 1)

plt.subplot(2, 3, 4)
sns.lineplot(data=ISD_df, x='Generation', y=rolling_tit_for_two_tats, color='#c84c54')
plt.xlabel('Generations')
plt.ylabel('Frequency')
plt.title('Tit-for-Two-Tats')
plt.grid(True)
plt.ylim(0, 1)

plt.subplot(2, 3, 5)
sns.lineplot(data=ISD_df, x='Generation', y=rolling_pavlov, color='#e08cc4')
plt.xlabel('Generations')
plt.ylabel('Frequency')
plt.title('Pavlov')
plt.grid(True)
plt.ylim(0, 1)

plt.tight_layout()  # Adjust layout for better spacing
plt.show()

*ISD Mean Strategy Frequencies*

In [ ]:
# Calculate mean strategy frequencies
ISD_mean_always_cooperate_freq = ISD_df['always_cooperate_frequency'].mean()
ISD_mean_always_defect_freq = ISD_df['always_defect_frequency'].mean()
ISD_mean_tit_for_tat_freq = ISD_df['tit_for_tat_frequency'].mean()
ISD_mean_tit_for_two_tats_freq = ISD_df['tit_for_two_tats_frequency'].mean()
ISD_mean_pavlov_freq = ISD_df['pavlov_frequency'].mean()

# Print the mean values
print("Mean always cooperate frequency:", ISD_mean_always_cooperate_freq)
print("Mean always defect frequency:", ISD_mean_always_defect_freq)
print("Mean tit for tat frequency:", ISD_mean_tit_for_tat_freq)
print("Mean tit for two tats frequency:", ISD_mean_tit_for_two_tats_freq)
print("Mean pavlov frequency:", ISD_mean_pavlov_freq)


*ISD Spearman's Rank Correlation between frequency of cooperation and mean intelligence*

In [ ]:
# Spearman's Rank Correlation for frequency of cooperative acts and mean intelligence
# Compute the correlation coefficient (rho) and p-value
ISD_intelligence_cooperation_rho, ISD_intelligence_cooperation_pvalue = spearmanr(ISD_cooperation_frequency, ISD_mean_intelligence)

print("Spearman's Rank Correlation:")
print("rho:", ISD_intelligence_cooperation_rho)
print("p-value:", ISD_intelligence_cooperation_pvalue)

In [ ]:
# Calculate the LOWESS line
lowess = sm.nonparametric.lowess(ISD_cooperation_frequency, ISD_mean_intelligence)

plt.figure(figsize=(10, 5))
sns.scatterplot(x=ISD_mean_intelligence, y=ISD_cooperation_frequency, alpha=0.5)
plt.plot(lowess[:, 0], lowess[:, 1], linewidth=2)  # Plot the LOESS line
plt.xlabel('Mean Intelligence')
plt.ylabel('Frequency of Cooperation')
plt.title('Correlation between Frequency of Cooperation and Mean Intelligence')
plt.grid(True)
plt.show()

*ISD Selection for Intelligence*

In [ ]:
# Calculate the covariance between fitness and intelligence in each generation
ISD_covariance_values = []
for fitness_values, intelligence_values in zip(ISD_df['ISD_network_fitness_values'], ISD_df['ISD_network_intelligence_values']):
    cov = np.cov(fitness_values, intelligence_values)[0][1]
    ISD_covariance_values.append(cov)

# Divide by the mean fitness in the population
ISD_selection_for_intelligence = [cov / mean_fitness for cov, mean_fitness in zip(ISD_covariance_values, ISD_mean_fitness)]

# Add a column in the pandas dataframe for the selection for intelligence measure each generation.
ISD_df['ISD_selection_for_intelligence'] = ISD_selection_for_intelligence

In [ ]:
plt.figure(figsize=(10, 5))
sns.lineplot(data = ISD_df, x= 'Generation', y= 'ISD_selection_for_intelligence')
plt.xlabel('Generations')
plt.ylabel('Selection for Intelligence')
plt.title('Selection for Intelligence over Generations')
plt.grid(True)
plt.show()

In [ ]:
# Mean selection for intelligence
print(ISD_df['ISD_selection_for_intelligence'].mean())

*ISD Correlation between Selection for Intelligence and Strategy Frequencies*

In [ ]:
# Spearman's Rank Correlation for selection of intelligence and strategy frequencies
ISD_ac_selection_rho, ISD_ac_selection_pvalue = spearmanr(ISD_df['always_cooperate_frequency'], ISD_df['ISD_selection_for_intelligence'])
ISD_ad_selection_rho, ISD_ad_selection_pvalue = spearmanr(ISD_df['always_defect_frequency'], ISD_df['ISD_selection_for_intelligence'])
ISD_tft_selection_rho, ISD_tft_selection_pvalue = spearmanr(ISD_df['tit_for_tat_frequency'], ISD_df['ISD_selection_for_intelligence'])
ISD_tftt_selection_rho, ISD_tftt_selection_pvalue = spearmanr(ISD_df['tit_for_two_tats_frequency'], ISD_df['ISD_selection_for_intelligence'])
ISD_pvlv_selection_rho, ISD_pvlv_selection_pvalue = spearmanr(ISD_df['pavlov_frequency'], ISD_df['ISD_selection_for_intelligence'])

# Print results for each strategy
print("Always Cooperate:")
print("rho:", ISD_ac_selection_rho)
print("p-value:", ISD_ac_selection_pvalue)
print("")
print("Always Defect:")
print("rho:", ISD_ad_selection_rho)
print("p-value:", ISD_ad_selection_pvalue)
print("")
print("Tit for tat:")
print("rho:", ISD_tft_selection_rho)
print("p-value:", ISD_tft_selection_pvalue)
print("")
print("Tit for two tats:")
print("rho:", ISD_tftt_selection_rho)
print("p-value:", ISD_tftt_selection_pvalue)
print("")
print("Pavlov:")
print("rho:", ISD_pvlv_selection_rho)
print("p-value:", ISD_pvlv_selection_pvalue)